# 빅데이터분석기사 8주차 모의고사 2회

### 데이터
- 1번 문제 : shipping_X_train / shipping_y_train / shipping_X_test / shipping_submission
- 2번 문제 : clients_train / clients_test / clients_submission

## 1-1. Warehouse_block 별로 Cost_of_the_Product와 Weight_in_gms의 상관계수를 구하고 두번째, 네번째로 상관관계가 강한 값들의 곱을 출력하시오.

In [3]:
import pandas as pd
import numpy as np

In [84]:
X_train = pd.read_csv("shipping_X_train.csv")
y_train = pd.read_csv("shipping_y_train.csv")
test = pd.read_csv("shipping_X_test.csv")
submission = pd.read_csv("shipping_submission.csv")

In [85]:
X_train.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6045,A,Flight,4,3,266,5,high,F,5,1590
1,44,F,Ship,3,1,174,2,low,M,44,1556
2,7940,F,Road,4,1,154,10,high,M,10,5674
3,1596,F,Ship,4,3,158,3,medium,F,27,1207
4,4395,A,Flight,5,3,175,3,low,M,7,4833


In [86]:
X_train.Warehouse_block.unique()

array(['A', 'F', 'C', 'B', 'D'], dtype=object)

In [87]:
a = X_train[X_train.Warehouse_block == 'A']
b = X_train[X_train.Warehouse_block == 'B']
c = X_train[X_train.Warehouse_block == 'C']
d = X_train[X_train.Warehouse_block == 'D']
f = X_train[X_train.Warehouse_block == 'F']

Cost_of_the_Product와 Weight_in_gms

In [88]:
a_c = a.corr()['Cost_of_the_Product']['Weight_in_gms']
b_c = b.corr()['Cost_of_the_Product']['Weight_in_gms']
c_c = c.corr()['Cost_of_the_Product']['Weight_in_gms']
d_c = d.corr()['Cost_of_the_Product']['Weight_in_gms']
f_c = f.corr()['Cost_of_the_Product']['Weight_in_gms']

In [89]:
a_c, b_c, c_c, d_c, f_c

(-0.14148106808904823,
 -0.08054612765484861,
 -0.1361480385913863,
 -0.1606446568710385,
 -0.15677176112946)

In [90]:
c_c, f_c

(-0.1361480385913863, -0.15677176112946)

In [91]:
print(c_c * f_c)

0.021344167784293315


## 1-2. Cost_of_the_Product를 표준화한 결과 중 -0.25 초과, 0.4 미만인 데이터의 Customer_rating 중위값과 Cost_of_the_Product를 정규화한 결과 중 0.1 이상, 0.6 이하인 데이터의 Customer_care_calls의 평균을 곱한 값을 출력하시오.

In [92]:
X_train.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6045,A,Flight,4,3,266,5,high,F,5,1590
1,44,F,Ship,3,1,174,2,low,M,44,1556
2,7940,F,Road,4,1,154,10,high,M,10,5674
3,1596,F,Ship,4,3,158,3,medium,F,27,1207
4,4395,A,Flight,5,3,175,3,low,M,7,4833


표준화한 결과 중 -0.25 초과, 0.4 미만

In [93]:
from sklearn.preprocessing import StandardScaler

In [94]:
ss = StandardScaler()

In [95]:
X_train['ss_cp'] = ss.fit_transform(X_train[['Cost_of_the_Product']])

In [96]:
result1 = X_train.query('-0.25 < ss_cp < 0.4')

In [97]:
value1 = result1.Customer_rating.median()

In [98]:
value1

3.0

정규화한 결과 중 0.1 이상, 0.6 이하

In [99]:
from sklearn.preprocessing import MinMaxScaler

In [100]:
ms = MinMaxScaler()

In [101]:
X_train['ms_cp'] = ms.fit_transform(X_train[['Cost_of_the_Product']])

In [102]:
result2 = X_train.query('0.1 <= ms_cp <= 0.6')

In [103]:
value2 = result2.Customer_rating.mean()

In [104]:
value2

2.9735955056179777

In [105]:
print(value1 * value2)

8.920786516853934


## 1-3. Reached.on.Time_Y.N을 예측하는 모델링을 수행하시오. 예측확률 값을 submission형태로 제출하고 이 때 파일명을 result1으로 저장하시오.

In [106]:
X_train.drop(['ID', 'ms_cp', 'ss_cp'], axis = 1, inplace = True)
test.drop('ID', axis = 1, inplace = True)

In [107]:
train = pd.concat([X_train, y_train[['Reached.on.Time_Y.N']]], axis = 1)

In [108]:
train['Gender'] = [1 if g == 'M' else 0 for g in train['Gender']]
test['Gender'] = [1 if g == 'M' else 0 for g in test['Gender']]

In [110]:
train.Customer_care_calls.unique()

array(['4', '3', '5', '6', '2', '$7'], dtype=object)

#### 정규 표현식 이용

In [78]:
import re

In [81]:
train['Customer_care_calls'] = [int(re.sub(r'[^0-9]+', '', s)) for s in train['Customer_care_calls']]

In [83]:
test['Customer_care_calls'] = [int(re.sub(r'[^0-9]+', '', s)) for s in test['Customer_care_calls']]

In [82]:
train.Customer_care_calls.unique()

array([4, 3, 5, 6, 2, 7])

#### str.replace 사용

In [111]:
train['Customer_care_calls'] = train['Customer_care_calls'].str.replace('$', '').astype(int)

<ipython-input-111-a18334066eec>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  train['Customer_care_calls'] = train['Customer_care_calls'].str.replace('$', '').astype(int)


In [112]:
test['Customer_care_calls'] = test['Customer_care_calls'].str.replace('$', '').astype(int)

<ipython-input-112-47d831329eaf>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  test['Customer_care_calls'] = test['Customer_care_calls'].str.replace('$', '').astype(int)


In [113]:
train.Customer_care_calls.unique()

array([4, 3, 5, 6, 2, 7])

In [114]:
X = train.drop('Reached.on.Time_Y.N', axis = 1)
y = train['Reached.on.Time_Y.N']

In [115]:
X = pd.get_dummies(X)

In [118]:
target = pd.get_dummies(test)

In [119]:
target.shape, X.shape

((4401, 18), (6598, 18))

In [120]:
target = target[X.columns]

### SVM, RandomForest

In [121]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

### Hold Out

In [124]:
from sklearn.model_selection import train_test_split

In [125]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [126]:
svc = SVC(probability = True)

In [127]:
svc.fit(X_train, y_train)

SVC(probability=True)

In [128]:
svc_pred = svc.predict_proba(X_val)[:, 1]

In [130]:
from sklearn.metrics import roc_auc_score

In [132]:
svc_score = roc_auc_score(y_val, svc_pred)

In [133]:
svc_score

0.7286911759093164

In [134]:
rf = RandomForestClassifier(random_state = 42)

In [135]:
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [136]:
rf_pred = rf.predict_proba(X_val)[:, 1]

In [137]:
rf_score = roc_auc_score(y_val, rf_pred)

In [138]:
rf_score

0.7516733617037519

### 전체 데이터 학습

In [143]:
svc = SVC(probability = True)

In [144]:
svc.fit(X, y)

SVC(probability=True)

In [145]:
svc_pred = svc.predict_proba(target)[:, 1]

In [146]:
svc_pred

array([0.73545943, 0.34606237, 0.59951557, ..., 0.37195058, 0.34324547,
       0.79621129])

In [140]:
rf.fit(X, y)

RandomForestClassifier(random_state=42)

In [141]:
rf_pred = rf.predict_proba(target)[:, 1]

In [142]:
rf_pred

array([0.43, 0.46, 0.44, ..., 0.35, 0.31, 1.  ])

In [147]:
submission['Reached.on.Time_Y.N'] = rf_pred * 0.65 + svc_pred * 0.35

In [148]:
submission.head()

,ID,Reached.on.Time_Y.N
0,6811,0.536911
1,4320,0.420122
2,5732,0.495830
3,7429,0.591969
4,2191,0.931595


In [149]:
#submission.to_csv('result1.csv', index = False)

### Ensemble

In [150]:
from sklearn.model_selection import StratifiedKFold

In [151]:
skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

In [152]:
svc_pred = np.zeros(target.shape[0])
scores = []
for tr_idx, val_idx in skf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    svc = SVC(probability = True)
    svc.fit(tr_x, tr_y)
    
    val_pred = svc.predict_proba(val_x)[:, 1]
    val_score = roc_auc_score(val_y, val_pred)
    scores.append(val_score)
    
    fold_pred = svc.predict_proba(target)[:, 1] / 10
    svc_pred += fold_pred

In [153]:
np.mean(scores)

0.7256092173863691

#### RandomForest Ensemble

In [154]:
rf_pred = np.zeros(target.shape[0])
scores = []
for tr_idx, val_idx in skf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    rf = RandomForestClassifier(random_state = 42)
    rf.fit(tr_x, tr_y)
    
    val_pred = rf.predict_proba(val_x)[:, 1]
    val_score = roc_auc_score(val_y, val_pred)
    scores.append(val_score)
    
    fold_pred = rf.predict_proba(target)[:, 1] / 10
    rf_pred += fold_pred

In [155]:
np.mean(scores)

0.7422187988399715

In [156]:
submission['Reached.on.Time_Y.N'] = rf_pred * 0.75 + svc_pred * 0.25

In [158]:
submission.head()

,ID,Reached.on.Time_Y.N
0,6811,0.515248
1,4320,0.399009
2,5732,0.482839
3,7429,0.529212
4,2191,0.950521


In [159]:
#submission.to_csv('result1.csv', index = False)

### Parameter Tuning

In [160]:
from sklearn.model_selection import GridSearchCV

In [169]:
rf_params = {'max_depth' : [4, 5, 6], 'n_estimators' : [100, 300]}

In [170]:
rf_grid = GridSearchCV(rf, rf_params, scoring = 'roc_auc', cv = skf)

10 Fold * max_depth(3) * n_estimators(2) ---> fitting 60!!!

In [171]:
rf_grid.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [4, 5, 6], 'n_estimators': [100, 300]},
             scoring='roc_auc')

In [172]:
rf_grid.best_estimator_

RandomForestClassifier(max_depth=4, n_estimators=300, random_state=42)

In [173]:
rf_grid.best_params_

{'max_depth': 4, 'n_estimators': 300}

In [175]:
rf_grid.best_score_

0.7517393459572195

***
## 2-1. age를 기준으로 이상치를 탐색하고 이상치 데이터의 higher education 비율을 출력하시오.

In [177]:
train = pd.read_csv('clients_train.csv')
test = pd.read_csv('clients_test.csv')
submission = pd.read_csv('client_submission.csv')

In [179]:
q1 = train['age'].quantile(0.25)
q3 = train['age'].quantile(0.75)

In [180]:
q1, q3

(26.0, 44.0)

In [181]:
q1 = np.percentile(train['age'], 25)
q3 = np.percentile(train['age'], 75)

In [182]:
q1, q3

(26.0, 44.0)

In [183]:
iqr = q3 - q1

In [184]:
l_b = q1 - 1.5 * iqr
u_b = q3 + 1.5 * iqr

In [186]:
out = train[train.age > u_b]

In [188]:
out.education.unique()

array(['Secondary education', 'Secondary special education',
       'Higher education'], dtype=object)

In [191]:
print(out[out.education == 'Higher education'].shape[0] / out.shape[0])

0.4


In [196]:
out.education.value_counts()

Higher education               10
Secondary education             8
Secondary special education     7
Name: education, dtype: int64

In [195]:
print(out.education.value_counts()['Higher education'])
print(out.education.value_counts().sum())

10
25


## 2-2. income에 대해 정규화를 실시하고 결과값이 0.3 이상 0.65미만인 데이터의 credit_amount와 age의 상관계수를 출력하시오.

In [198]:
from sklearn.preprocessing import MinMaxScaler

In [199]:
ms = MinMaxScaler()

In [200]:
train['ms_i'] = ms.fit_transform(train[['income']])

In [202]:
result = train.query('0.3 <= ms_i < 0.65')

#### numpy 이용

In [205]:
np.corrcoef(result['credit_amount'], result['age'])[0, 1]

-0.03882987309782206

#### pandas 이용

In [207]:
result.corr()['credit_amount']['age']

-0.03882987309782211

## 2-3. product_type별로 credit_amount의 왜도를 탐색하고 이 때 3, 5번째로 왜도의 절댓값이 큰 type을 리스트로 출력하시오.

In [217]:
result = abs(train.groupby('product_type').credit_amount.skew()).sort_values()[::-1].reset_index()

In [223]:
result

,product_type,credit_amount
0,Music,NaN
1,Garden equipment,NaN
2,Childen's goods,NaN
3,Furniture,2.835442
4,Windows & Doors,2.626691
5,Household appliances,2.481810
6,Clothing,2.287323
7,Cosmetics and beauty services,2.117954
8,Auto,2.073833
9,Jewelry,1.699410


In [222]:
print(result.iloc[[5, 7]].product_type.tolist())

['Household appliances', 'Cosmetics and beauty services']


In [213]:
answer = ['Household appliances', 'Cosmetics and beauty services']

In [214]:
print(answer)

['Household appliances', 'Cosmetics and beauty services']


## 2-4. bad_client_target을 예측하는 모델링을 수행하시오. 이 때 submission의 bad_client_target을 확률값으로 도출하고 result3 파일명으로 저장하시오.

In [225]:
train.drop('ms_i', axis = 1, inplace = True)

In [227]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [339]:
X = train.drop('bad_client_target', axis = 1)
y = train.bad_client_target

target = test.drop('bad_client_target', axis = 1)

In [340]:
X['sex'] = [1 if s == 'female' else 0 for s in X['sex']]
target['sex'] = [1 if s == 'female' else 0 for s in target['sex']]

In [341]:
train.groupby('education').bad_client_target.mean().sort_values()

education
Incomplete secondary education    0.000000
PhD degree                        0.000000
Higher education                  0.058939
Incomplete higher education       0.068493
Secondary special education       0.131215
Secondary education               0.181818
Name: bad_client_target, dtype: float64

In [342]:
def new_edu(x) :
    if x in ['Higher education', 'Incomplete higher education'] :
        return 2
    elif x in ['Secondary education', 'Secondary special education'] :
        return 3
    else :
        return 1

In [343]:
X['education'] = X['education'].apply(new_edu)

In [344]:
target['education'] = target['education'].apply(new_edu)

In [345]:
def new_type(x) :
    if x in ['Audio & Video', 'Sporting goods', 'Music', 'Garden equipment',
       "Childen's goods", 'Training', 'Fitness'] :
        return 1
    elif x in ['Furniture', 'Clothing', 'Windows & Doors', 'Medical services',
       'Computers', 'Auto', 'Tourism', 'Household appliances',
       'Cosmetics and beauty services'] :
        return 2
    else :
        return 3

In [346]:
X['product_type'] = X['product_type'].apply(new_type)

In [347]:
target['product_type'] = target['product_type'].apply(new_type)

In [348]:
X = pd.get_dummies(X)
target = pd.get_dummies(target)

In [349]:
target = target[X.columns]

In [350]:
from sklearn.preprocessing import StandardScaler

In [351]:
ss = StandardScaler()

In [352]:
X = pd.DataFrame(ss.fit_transform(X), columns = X.columns)

In [353]:
target = pd.DataFrame(ss.transform(target), columns = X.columns)

In [354]:
lr_model = LogisticRegression(random_state = 42)
dtc = DecisionTreeClassifier(random_state = 42)

In [355]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 42, stratify = y, test_size = 0.25)

In [298]:
lr_model.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [299]:
lr_pred = lr_model.predict_proba(X_val)[:, 1]

In [300]:
lr_score = roc_auc_score(y_val, lr_pred)

In [301]:
lr_score

0.7690959544325982

In [302]:
dtc.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [303]:
dtc_pred = dtc.predict_proba(X_val)[:, 1]

In [304]:
dtc_score = roc_auc_score(y_val, dtc_pred)

In [305]:
dtc_score

0.5682780779903608

#### 전체 데이터 학습

In [306]:
lr_model.fit(X, y)

LogisticRegression(random_state=42)

In [307]:
lr_pred = lr_model.predict_proba(target)[:, 1]

In [309]:
submission['bad_client_target'] = lr_pred

In [310]:
submission.head()

,index,bad_client_target
0,0,0.067311
1,1,0.047440
2,2,0.184097
3,3,0.213726
4,4,0.496248


In [311]:
#submission.to_csv("result3.csv", index = False)

### Parameter Tuning

In [313]:
skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)

In [312]:
params = {'max_depth' : [3, 4, 5, 6], 'min_samples_split' : [2, 4, 8]}

In [315]:
dtc_grid = GridSearchCV(dtc, scoring = 'roc_auc', cv = skf, param_grid = params)

In [316]:
dtc_grid.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [3, 4, 5, 6],
                         'min_samples_split': [2, 4, 8]},
             scoring='roc_auc')

In [317]:
dtc_grid.best_estimator_

DecisionTreeClassifier(max_depth=3, random_state=42)

In [325]:
dtc_grid.best_score_

0.7269135071366015

In [319]:
best_params = dtc_grid.best_params_

In [320]:
dtc = DecisionTreeClassifier(random_state = 42, **best_params)

In [321]:
dtc.fit(X, y)

DecisionTreeClassifier(max_depth=3, random_state=42)

In [322]:
dtc_pred = dtc.predict_proba(target)[:, 1]

In [326]:
submission['bad_client_target'] = dtc_pred * 0.2 + lr_pred * 0.8

In [327]:
submission.head()

,index,bad_client_target
0,0,0.070671
1,1,0.054774
2,2,0.192786
3,3,0.188581
4,4,0.513127


In [328]:
#submission.to_csv('result3.csv', index = False)

### Except

In [333]:
from sklearn.ensemble import RandomForestClassifier

In [334]:
rf = RandomForestClassifier(random_state = 42)

In [335]:
params = {'max_depth' : np.arange(3, 100), 'n_estimators' : [750, 800, 1000, 1200]} 

Fitting 수 : 파라미터1의 범위 * 파라미터 2의 범위 * ---- * 파라미터 n의 범위 * kfold의 k수

In [336]:
rf_grid = GridSearchCV(rf, param_grid = params, cv = skf, scoring = 'roc_auc')

In [356]:
rf_grid.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [3, 4, 5],
                         'n_estimators': [100, 500, 1000]},
             scoring='roc_auc')

In [357]:
rf_grid.best_params_

{'max_depth': 4, 'n_estimators': 1000}

***

In [260]:
t1 = pd.DataFrame({'city' : ['서울', '도쿄', '파리']})

In [263]:
pd.get_dummies(t1)

,city_도쿄,city_서울,city_파리
0,0,1,0
1,1,0,0
2,0,0,1


In [264]:
t2 = pd.DataFrame({'city' : ['서울', '도쿄']})

In [268]:
t2 = pd.get_dummies(t2)

In [270]:
t2['city_파리'] = 0

In [271]:
t2

,city_도쿄,city_서울,city_파리
0,0,1,0
1,1,0,0
